In [15]:
#packages
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.ensemble import ExtraTreesClassifier


In [120]:
#data
dna=pd.read_excel('dnamethylation.xlsx')
rna=pd.read_excel('RNAseq.xlsx')
tcga=pd.read_csv('TCGA-PRAD.GDC_phenotype.tsv',sep='\t')
dna=dna.T
rna=rna.T
newheader=dna.iloc[0]
dna.columns=newheader
dna.drop(index=dna.index[0], axis=0, inplace=True)
tcga.head()



,submitter_id.samples,additional_pharmaceutical_therapy,additional_radiation_therapy,age_at_initial_pathologic_diagnosis,batch_number,bcr,bcr_followup_barcode,bcr_followup_uuid,submitter_id,biochemical_recurrence,...,intermediate_dimension.samples,is_ffpe.samples,longest_dimension.samples,oct_embedded.samples,preservation_method.samples,sample_type.samples,sample_type_id.samples,shortest_dimension.samples,state.samples,tissue_type.samples
0,TCGA-J9-A8CN-01A,NaN,NaN,53,357.55.0,Nationwide Children's Hospital,NaN,NaN,TCGA-J9-A8CN,NO,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
1,TCGA-V1-A8MM-01A,NO,YES,60,389.51.0,Nationwide Children's Hospital,TCGA-V1-A8MM-F69416,B309B20B-78D8-4EEA-AF82-B78EA611F897,TCGA-V1-A8MM,YES,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
2,TCGA-EJ-A46H-01A,NaN,NaN,60,285.66.0,Nationwide Children's Hospital,TCGA-EJ-A46H-F64713,3A0C7921-3A63-4F16-946C-34C90EEFB542,TCGA-EJ-A46H,NO,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported
3,TCGA-HC-7078-01A,NaN,NaN,69,184.82.0,Nationwide Children's Hospital,TCGA-HC-7078-F47785,F203FC37-1D0A-4D4C-9572-C0E34E8C13BD,TCGA-HC-7078,NaN,...,0.5,False,0.9,NaN,NaN,Primary Tumor,1,0.1,released,Not Reported
4,TCGA-VN-A88K-01A,NaN,NaN,58,357.55.0,Nationwide Children's Hospital,TCGA-VN-A88K-F62590,B3F4B606-68A9-4D0E-8928-AEBC54A82A82,TCGA-VN-A88K,NO,...,NaN,False,NaN,True,NaN,Primary Tumor,1,NaN,released,Not Reported


In [37]:
dna.head()

Composite Element REF,rs798149,rs5931272,rs1416770,rs5987737,rs2521373,rs6626309,rs5936512,cg27547703,rs5926356,cg11738485,...,cg12079322,cg22630918,rs654498,cg14699728,cg11751645,cg21135483,cg05972871,cg09491991,cg00349607,cg27069588
TCGA-G9-6364-11A,0.973926,0.017786,0.034421,0.962181,0.9426,0.049391,0.035194,0.955048,0.101058,0.022476,...,0.03677,0.911938,0.508638,0.069513,0.329193,0.982722,0.012397,0.017325,0.079807,0.940124
TCGA-EJ-5505-01A,0.017878,0.974465,0.038245,0.03226,0.923651,0.914774,0.048662,0.943396,0.910735,0.024471,...,0.153523,0.72646,0.503856,0.66229,0.496958,0.890954,0.585969,0.648755,0.668784,0.780054
TCGA-ZG-A9L6-01A,0.9714,0.012692,0.056513,0.940169,0.924574,0.899239,0.07845,0.618288,0.710405,0.50209,...,0.043859,0.741753,0.440344,0.513371,0.520829,0.666798,0.45512,0.526718,0.479927,0.640653
TCGA-HC-A8D0-01A,0.017656,0.963777,0.948281,0.053624,0.080221,0.890946,0.053176,0.22636,0.212449,0.537972,...,0.088295,0.797301,0.897439,0.044651,0.353558,0.916566,0.041186,0.03193,0.464376,0.899074
TCGA-XK-AAK1-01A,0.014043,0.015018,0.037652,0.045543,0.93332,0.077728,0.060646,0.941907,0.208142,0.972591,...,0.594681,0.767568,0.508172,0.719555,0.881002,0.974659,0.687865,0.568145,0.821149,0.303898


In [125]:
newheader2=rna.iloc[0]
rna.columns=newheader2
rna.drop(index=rna.index[0], axis=0, inplace=True)

rna.head()

Ensembl_ID,ENSG00000229314.5,ENSG00000163810.10,ENSG00000226278.1,ENSG00000265933.4,ENSG00000124233.11,ENSG00000092054.12,ENSG00000124157.6,ENSG00000096006.10,ENSG00000139219.16,ENSG00000184956.14,...,ENSG00000166828.2,ENSG00000148584.13,ENSG00000169752.15,ENSG00000181143.14,ENSG00000183117.16,ENSG00000204767.3,ENSG00000250846.5,ENSG00000274835.1,ENSG00000255545.6,ENSG00000238178.5
TCGA-EJ-5505-01A,11.630267,8.909893,0.0,14.086551,3.321928,0.0,3.169925,6.442943,3.584963,3.0,...,8.503826,9.383704,10.986553,4.321928,6.930737,5.044394,4.523562,4.954196,10.543032,6.78136
TCGA-ZG-A9L6-01A,6.72792,6.044394,0.0,9.485829,0.0,2.321928,0.0,4.523562,4.247928,1.0,...,6.807355,3.906891,7.787903,2.321928,3.584963,4.906891,6.988685,3.169925,8.413628,5.247928
TCGA-HC-A8D0-01A,6.584963,16.373477,8.861087,4.247928,3.321928,1.0,2.321928,14.634868,4.459432,6.72792,...,9.751544,2.0,5.087463,7.851749,5.285402,4.643856,3.807355,1.0,9.954196,5.247928
TCGA-XK-AAK1-01A,0.0,3.459432,0.0,2.321928,0.0,0.0,0.0,6.643856,2.321928,2.807355,...,6.554589,1.0,6.321928,4.087463,2.807355,0.0,0.0,2.584963,7.276124,3.584963
TCGA-VN-A88I-01A,5.459432,9.308339,1.0,11.136991,8.247928,13.20503,6.72792,3.807355,4.754888,5.426265,...,8.965784,0.0,4.754888,7.686501,2.584963,3.0,4.169925,1.0,7.375039,3.169925


In [127]:
dnamerge=dna.merge(tcga[['submitter_id.samples','sample_type.samples']],left_index=True,right_on='submitter_id.samples',how='left')
rnamerge=rna.merge(dnamerge,left_index=True,right_on='submitter_id.samples')
rnamerge.head()

,ENSG00000229314.5,ENSG00000163810.10,ENSG00000226278.1,ENSG00000265933.4,ENSG00000124233.11,ENSG00000092054.12,ENSG00000124157.6,ENSG00000096006.10,ENSG00000139219.16,ENSG00000184956.14,...,rs654498,cg14699728,cg11751645,cg21135483,cg05972871,cg09491991,cg00349607,cg27069588,submitter_id.samples,sample_type.samples
31,11.630267,8.909893,0.0,14.086551,3.321928,0.0,3.169925,6.442943,3.584963,3.0,...,0.503856,0.66229,0.496958,0.890954,0.585969,0.648755,0.668784,0.780054,TCGA-EJ-5505-01A,Primary Tumor
188,6.72792,6.044394,0.0,9.485829,0.0,2.321928,0.0,4.523562,4.247928,1.0,...,0.440344,0.513371,0.520829,0.666798,0.45512,0.526718,0.479927,0.640653,TCGA-ZG-A9L6-01A,Primary Tumor
11,6.584963,16.373477,8.861087,4.247928,3.321928,1.0,2.321928,14.634868,4.459432,6.72792,...,0.897439,0.044651,0.353558,0.916566,0.041186,0.03193,0.464376,0.899074,TCGA-HC-A8D0-01A,Primary Tumor
77,0.0,3.459432,0.0,2.321928,0.0,0.0,0.0,6.643856,2.321928,2.807355,...,0.508172,0.719555,0.881002,0.974659,0.687865,0.568145,0.821149,0.303898,TCGA-XK-AAK1-01A,Primary Tumor
189,5.459432,9.308339,1.0,11.136991,8.247928,13.20503,6.72792,3.807355,4.754888,5.426265,...,0.533727,0.126584,0.078446,0.947461,0.072019,0.182851,0.294165,0.822068,TCGA-VN-A88I-01A,Primary Tumor


In [138]:
rnamerge.shape

(533, 2002)

In [130]:
rnamerge['sample_type.samples'].value_counts()

Primary Tumor          498
Solid Tissue Normal     35
Metastatic               1
Name: sample_type.samples, dtype: int64

In [131]:
rnamerge = rnamerge[rnamerge['sample_type.samples'] != 'Metastatic']
rnamerge['sample_type.samples'].unique() 

array(['Primary Tumor', 'Solid Tissue Normal'], dtype=object)

In [133]:
y=rnamerge['sample_type.samples']
y = label_binarize(
        y.values, classes=["Primary Tumor", "Solid Tissue Normal"]
    ).ravel()
id=rnamerge['submitter_id.samples']
x=rnamerge.drop(['submitter_id.samples','sample_type.samples'],axis=1)
x = x.astype('float')
x = x.fillna(x.median())
names=x.columns
names=pd.DataFrame(names)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123)
print(x_train.shape)
print(x_test.shape)

(426, 2000)
(107, 2000)


In [59]:
y_train.value_counts()

Primary Tumor          397
Solid Tissue Normal     44
Name: sample_type.samples, dtype: int64

In [134]:
#RFE
n=10
def RFEmethod(X,Y):
  rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=n)
  rfe.fit(X,Y)
  rankings=pd.DataFrame(rfe.ranking_) 
  ranked=pd.concat([names,rankings], axis=1)
  ranked.columns = ["Feature", "Rank"]
  print(ranked.sort_values(by=['Rank']))


In [135]:
RFEmethod(x_train.values,y_train) 


                 Feature  Rank
1475          cg07489502     1
1641          cg26896572     1
1642          cg09556042     1
1831          cg02032778     1
1830          cg07011961     1
...                  ...   ...
4     ENSG00000124233.11  1987
3      ENSG00000265933.4  1988
2      ENSG00000226278.1  1989
1     ENSG00000163810.10  1990
0      ENSG00000229314.5  1991

[2000 rows x 2 columns]


In [160]:
#univariate (f_regression)
def univariatemethod1(X,Y):
  model = SelectKBest(score_func=f_regression, k=n)
  results = model.fit(X, Y)
  results_df=pd.DataFrame(results.scores_)
  scored=pd.concat([names,results_df], axis=1)
  scored.columns = ["Feature", "Score"]
  print(scored.sort_values(by=['Score'],ascending=False))

#univariate (chi-squared)
from sklearn.feature_selection import chi2
def univariatemethod2(X,Y):
  model = SelectKBest(score_func=chi2, k=n)
  results = model.fit(X, Y)
  results_df=pd.DataFrame(results.scores_)
  scored=pd.concat([names,results_df], axis=1)
  scored.columns = ["Feature", "Score"]
  print(scored.sort_values(by=['Score'],ascending=False))



In [161]:
univariatemethod1(x_train.values,y_train)
univariatemethod2(x_train.values,y_train)



                 Feature       Score
1902          cg14117138  157.649024
1323          cg12539796  140.014526
1801          cg03514404  134.901491
1835          cg20883831  129.910893
1549          cg16647921  126.923077
...                  ...         ...
670   ENSG00000167858.11    0.000602
916    ENSG00000257884.2    0.000322
1120          rs10882854    0.000129
906   ENSG00000123201.13    0.000027
789    ENSG00000211625.2    0.000004

[2000 rows x 2 columns]
                 Feature       Score
933    ENSG00000171201.9  382.293851
154    ENSG00000167580.6  227.390501
4     ENSG00000124233.11  170.237081
230    ENSG00000171053.8  154.171890
686   ENSG00000122133.15  139.647178
...                  ...         ...
1890          cg21158163    0.000260
916    ENSG00000257884.2    0.000227
906   ENSG00000123201.13    0.000044
1120          rs10882854    0.000016
789    ENSG00000211625.2    0.000005

[2000 rows x 2 columns]


In [158]:
#Feature Importance
#Top Score is most important
def FeatureImportancemethod(X,Y):
  model = ExtraTreesClassifier()
  model.fit(X, Y)
  results_df=pd.DataFrame(model.feature_importances_)
  scored=pd.concat([names,results_df], axis=1)
  scored.columns = ["Feature", "Score"]
  print(scored.sort_values(by=['Score'],ascending=False))

In [159]:
FeatureImportancemethod(x_test,y_test)

                 Feature     Score
1095          cg22524657  0.013309
1820          cg02636041  0.013169
1619          cg03065447  0.012214
1139          cg02027945  0.012170
1767          cg10916998  0.012086
...                  ...       ...
758   ENSG00000166105.14  0.000000
757    ENSG00000226383.4  0.000000
756   ENSG00000169933.11  0.000000
755   ENSG00000150361.10  0.000000
1000            rs798149  0.000000

[2000 rows x 2 columns]
